In [2]:
%pip install transformers==4.29.0
%pip install --upgrade accelerate
%pip install datasets
%pip install mlflow
%pip install azureml-mlflow==1.52.0
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import mlflow
from azure.ai.ml import MLClient
from azureml.core import Workspace
from azure.identity import DefaultAzureCredential

subscription_id = '80c77c76-74ba-4c8c-8229-4c3b2957990c'
resource_group = 'sonata-test-rg'
workspace_name = 'sonata-test-ws'

credential = DefaultAzureCredential()
ws = Workspace(subscription_id, resource_group, workspace_name)

workspace_ml_client = MLClient(
        credential, subscription_id, resource_group, ws
    )

In [4]:
# store the mlflow results in azure ml workspace
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [5]:
modelname = 'gpt2-medium'
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForCausalLM.from_pretrained(modelname)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
basemodel = mlflow.transformers.log_model(
        transformers_model={"model": model, "tokenizer": tokenizer },
        task="text-generation",
        artifact_path="models/basemodel",
        input_example=["base model loaded directly from transformer's"],
        registered_model_name="Gpt-2-medium-base-model"
        )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023/07/17 15:06:34 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp6dfdvesm/model, flavor: transformers), fall back to return ['transformers==4.29.0', 'torch==2.0.1', 'torchvision==0.15.2', 'accelerate==0.21.0']. Set logging level to DEBUG to see the full traceback.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully re

In [42]:
mlmodel = mlflow.transformers.save_model(
    transformers_model={"model": model, "tokenizer": tokenizer },
    path="./gpt2savedmodel"
)

In [46]:
mlmodel

In [8]:
from datasets import load_dataset
traindataset = load_dataset("cnn_dailymail","3.0.0", split="train").shuffle().select(range(100))
testdataset = load_dataset("cnn_dailymail", "3.0.0", split="test").shuffle().select(range(50))

Found cached dataset cnn_dailymail (/home/azureuser/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
Found cached dataset cnn_dailymail (/home/azureuser/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


In [9]:
traindataset_df=traindataset.to_pandas()
traindataset_df

,article,highlights,id
0,"By . Chris Parsons . PUBLISHED: . 22:18 EST, 1...",Channel were 'disloyal and ungrateful' to X Fa...,8d9236aa44b87371d0c9ba71b497e9a453c36d40
1,Luay al-Khatteeb has spoken to CNN previously ...,Luay al-Khatteeb says ISIS now controls 60% of...,2c9f2c08ebfd60634f4f188c3af00d77f5da18dd
2,By . Hugo Gye and Peter Allen In Paris . PUBLI...,"Zaid Al-Hilli, 53, interrogated over killing o...",37cae83e10c72cf8705e0f9f8f2274cba041714d
3,Alex Salmond is planning a visit to watch the ...,4million Scots to take part in independence re...,bcebaa4f06b0f77b4bc176f26d4e9833066e8b7a
4,By . Pa Reporter . Jazz Carlin and Fran Halsal...,Jazz Carlin wins gold in 400m freestyle at the...,49fd21336d7632a1c7f00e83a5cdce5f83d5bfcb
...,...,...,...
95,(CNN) -- Jason Derulo has a message for fans w...,Derulo confirmed the breakup during a radio in...,acc6576067efe4b2204b6bd2b2010b59bb819af3
96,"By . Lucy Crossley . PUBLISHED: . 07:01 EST, 2...","Joy Turnbull, 63, was mending a fence with hus...",2d7c3c9573152d9b7368dbb04001b6ac581903b0
97,"By . Simon Parry . PUBLISHED: . 18:58 EST, 31 ...",Lindsay Sandiford did not find out about her f...,e405abf895265ad1c9cef9ad4e50c77ff6a96500
98,(CNN) -- More than two dozen children were amo...,"Twenty-seven minors, including one boy, were a...",5196ad6481251dbc29eb6a355013c12d2a3c2c5b


In [10]:
def tokenizer_function(examples):
        return tokenizer(examples["article"],text_target=examples["highlights"],truncation=True,padding=True,max_length=128,return_tensors="pt")

tokenizer.pad_token = tokenizer.eos_token
#tokenizer.add_special_tokens({'pad_token': '-'})
traindataset = traindataset.map(tokenizer_function,batched=True)
testdataset = testdataset.map(tokenizer_function,batched=True)

100%|██████████| 1/1 [00:00<00:00,  2.16ba/s]


In [11]:
traindataset_df=traindataset.to_pandas()
traindataset_df

,article,highlights,id,input_ids,attention_mask,labels
0,"By . Chris Parsons . PUBLISHED: . 22:18 EST, 1...",Channel were 'disloyal and ungrateful' to X Fa...,8d9236aa44b87371d0c9ba71b497e9a453c36d40,"[3886, 764, 5180, 35925, 764, 24676, 9148, 184...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[29239, 547, 705, 67, 3044, 4815, 290, 555, 21..."
1,Luay al-Khatteeb has spoken to CNN previously ...,Luay al-Khatteeb says ISIS now controls 60% of...,2c9f2c08ebfd60634f4f188c3af00d77f5da18dd,"[25596, 323, 435, 12, 42, 5183, 660, 1765, 468...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[25596, 323, 435, 12, 42, 5183, 660, 1765, 113..."
2,By . Hugo Gye and Peter Allen In Paris . PUBLI...,"Zaid Al-Hilli, 53, interrogated over killing o...",37cae83e10c72cf8705e0f9f8f2274cba041714d,"[3886, 764, 25930, 402, 5948, 290, 5613, 9659,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[57, 1698, 978, 12, 36369, 72, 11, 7192, 11, 4..."
3,Alex Salmond is planning a visit to watch the ...,4million Scots to take part in independence re...,bcebaa4f06b0f77b4bc176f26d4e9833066e8b7a,"[15309, 4849, 6327, 318, 5410, 257, 3187, 284,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19, 14100, 33860, 284, 1011, 636, 287, 10404,..."
4,By . Pa Reporter . Jazz Carlin and Fran Halsal...,Jazz Carlin wins gold in 400m freestyle at the...,49fd21336d7632a1c7f00e83a5cdce5f83d5bfcb,"[3886, 764, 11243, 25869, 764, 21406, 1879, 28...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[41, 8101, 1879, 2815, 7864, 3869, 287, 7337, ..."
...,...,...,...,...,...,...
95,(CNN) -- Jason Derulo has a message for fans w...,Derulo confirmed the breakup during a radio in...,acc6576067efe4b2204b6bd2b2010b59bb819af3,"[7, 18474, 8, 1377, 8982, 9626, 43348, 468, 25...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[28532, 43348, 4999, 262, 41584, 1141, 257, 52..."
96,"By . Lucy Crossley . PUBLISHED: . 07:01 EST, 2...","Joy Turnbull, 63, was mending a fence with hus...",2d7c3c9573152d9b7368dbb04001b6ac581903b0,"[3886, 764, 22162, 6372, 1636, 764, 24676, 914...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[41338, 26445, 11, 8093, 11, 373, 285, 1571, 2..."
97,"By . Simon Parry . PUBLISHED: . 18:58 EST, 31 ...",Lindsay Sandiford did not find out about her f...,e405abf895265ad1c9cef9ad4e50c77ff6a96500,"[3886, 764, 11288, 2547, 563, 764, 24676, 9148...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[43410, 16706, 3837, 361, 585, 750, 407, 1064,..."
98,(CNN) -- More than two dozen children were amo...,"Twenty-seven minors, including one boy, were a...",5196ad6481251dbc29eb6a355013c12d2a3c2c5b,"[7, 18474, 8, 1377, 3125, 621, 734, 8667, 1751...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[34096, 12, 26548, 21423, 11, 1390, 530, 2933,..."


In [12]:
traindataset = traindataset.remove_columns(["article","highlights","id"])
testdataset = testdataset.remove_columns(["article","highlights","id"])

In [13]:
import evaluate
metrics = evaluate.load("perplexity")

In [14]:
import numpy as np
def compute_metrics(eval_period):
        print(eval_period) 
        logits, labels = eval_period
        predictions = np.argmax(logits,x_axis = 1)
        return metrics.compute(predictions = predictions, references = labels)

In [49]:
from transformers import TrainingArguments,Trainer,TrainerCallback

class MyCallback(TrainerCallback):
    "A callback that prints a message at the beginning of training"

    def on_train_begin(self, args, state, control, **kwargs):
        print("Starting training")

Training_args = TrainingArguments(
        num_train_epochs = 1,
        output_dir= "output",
        save_strategy="epoch"
    )

trainer = Trainer(
        model = model,
        tokenizer=tokenizer,
        args = Training_args,
        compute_metrics=compute_metrics,
        train_dataset = traindataset,
        eval_dataset = testdataset,
        callbacks = [MyCallback]
    )

In [21]:
result = trainer.train()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Starting training


Step,Training Loss


In [23]:
result.metrics

{'train_runtime': 121.3436,
 'train_samples_per_second': 0.824,
 'train_steps_per_second': 0.107,
 'total_flos': 23217517363200.0,
 'train_loss': 6.043617835411658,
 'epoch': 1.0}